#FactorCL on Synthetic Dataset

In [1]:
!git clone https://github.com/pliang279/FactorCL

Cloning into 'FactorCL'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), 19.92 KiB | 1.99 MiB/s, done.


In [2]:
%cd FactorCL

/content/FactorCL


In [3]:
from Synthetic.dataset import*
from synthetic_models import*
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression

In [4]:
# Hyperparameters
A_dim, B_dim = 100, 100
x1_dim, x2_dim = 100, 100
y_dim = 1
label_dim = 1
#estimator = 'probabilistic_classifier'
lr = 1e-4
relative_ratio = 0.001
hidden_dim=512 
embed_dim=128
layers=1
activation = 'relu'

In [5]:

# Define custom dimensions of features and labels
feature_dim_info = dict()
label_dim_info = dict()

intersections = get_intersections(num_modalities=2)

feature_dim_info['12'] = 10
feature_dim_info['1'] = 6
feature_dim_info['2'] = 6

label_dim_info['12'] = 10
label_dim_info['1'] = 6
label_dim_info['2'] = 6

print(intersections)
print(feature_dim_info)
print(label_dim_info)

# Get datasets
total_data, total_labels, total_raw_features = generate_data(30000, 2, feature_dim_info, label_dim_info)
total_labels = get_labels(label_dim_info, total_raw_features)

dataset = MultimodalDataset(total_data, total_labels)

# Dataloader
batch_size = 256
num_data = total_labels.shape[0]
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8*num_data), num_data-int(0.8*num_data)])

train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True,
                            batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)
data_loader = DataLoader(dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)    


['1', '2', '12']
{'12': 10, '1': 6, '2': 6}
{'12': 10, '1': 6, '2': 6}


##FactorCL-SUP

In [13]:
#Train the model
rus_model = RUSModel(A_dim, B_dim, 20, hidden_dim, embed_dim).cuda()
train_rusmodel(rus_model, train_loader, dataset, num_epoch=10, num_club_iter=1)
rus_model.eval()

iter:  0  i_batch:  0  loss:  -4.513189196586609e-06
iter:  1  i_batch:  0  loss:  -3.0942587852478027
iter:  2  i_batch:  0  loss:  -5.861406326293945
iter:  3  i_batch:  0  loss:  -6.599287033081055
iter:  4  i_batch:  0  loss:  -6.948007583618164
iter:  5  i_batch:  0  loss:  -6.756012439727783
iter:  6  i_batch:  0  loss:  -6.895827293395996
iter:  7  i_batch:  0  loss:  -7.467083930969238
iter:  8  i_batch:  0  loss:  -7.240515232086182
iter:  9  i_batch:  0  loss:  -7.031285762786865


RUSModel(
  (backbones): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=128, bias=True)
    )
  )
  (linears_infonce_x1x2): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (linears_club_x1x2_cond): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (linea

In [14]:
# Simple evaluation using linear logistic regression model
# Embeddings
train_embeds = rus_model.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

test_embeds = rus_model.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)
print(score)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9861666666666666


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##SimCLR

In [15]:
simclr_model = SupConModel(A_dim, B_dim, hidden_dim, embed_dim, use_label=False).cuda()
simclr_optim = optim.Adam(simclr_model.parameters(), lr=lr)
train_supcon(simclr_model, train_loader, simclr_optim, num_epoch=10)
simclr_model.eval()

iter:  0  i_batch:  0  loss:  10.232044219970703
iter:  1  i_batch:  0  loss:  5.859925270080566
iter:  2  i_batch:  0  loss:  4.918781280517578
iter:  3  i_batch:  0  loss:  4.062376976013184
iter:  4  i_batch:  0  loss:  3.4061083793640137
iter:  5  i_batch:  0  loss:  2.7391912937164307
iter:  6  i_batch:  0  loss:  2.3390750885009766
iter:  7  i_batch:  0  loss:  1.6201176643371582
iter:  8  i_batch:  0  loss:  1.185381293296814
iter:  9  i_batch:  0  loss:  0.9489563703536987


SupConModel(
  (encoder_x1): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=128, bias=True)
  )
  (encoder_x2): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=128, bias=True)
  )
  (projection_x1): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
  )
  (projection_x2): Sequential(
    (0): Linear(in_features=128, out_features=1

In [17]:
train_embeds = simclr_model.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

# Embeddings
test_embeds = simclr_model.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)
print(score)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.963


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
